In [18]:
import os
import requests
import openai
import gradio as gr
from git import Repo
import shutil

In [19]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [20]:
# Function to validate GitHub repo and check if public
def is_valid_public_repo(repo_url):
    try:
        # Extract owner and repo name from the URL
        parts = repo_url.rstrip("/").split("/")
        owner, repo_name = parts[-2], parts[-1]
        api_url = f"https://api.github.com/repos/{owner}/{repo_name}"
        
        # Make GitHub API request
        response = requests.get(api_url)
        if response.status_code == 200:
            repo_info = response.json()
            if not repo_info.get("private"):
                return True, owner, repo_name
        return False, None, None
    except Exception as e:
        print(f"Error validating repo: {e}")
        return False, None, None

In [21]:
# Function to clone the GitHub repo locally
def clone_repo(owner, repo_name):
    repo_url = f"https://github.com/{owner}/{repo_name}.git"
    clone_dir = f"./temp_repo/{repo_name}"
    try:
        if os.path.exists(clone_dir):
            shutil.rmtree(clone_dir)  # Clean up if directory already exists
        Repo.clone_from(repo_url, clone_dir)
        return clone_dir
    except Exception as e:
        print(f"Error cloning repo: {e}")
        return None


In [28]:
# Function to analyze a code file and suggest comments using GPT-4
def suggest_comments_for_file(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            code = f.read()
        
        # Prompt for GPT-4
        prompt = f"Analyze the following code and suggest comments/docstrings:\n{code}"
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        comments = response.choices[0].message.content
        return comments
    except Exception as e:
        print(f"Error analyzing file {file_path}: {e}")
        return None

In [29]:
def process_repo(repo_url):
    is_valid, owner, repo_name = is_valid_public_repo(repo_url)
    if not is_valid:
        return "Invalid or private repository. Please provide a valid public GitHub repo link."
    
    # Clone repo
    repo_path = clone_repo(owner, repo_name)
    if not repo_path:
        return "Failed to clone the repository."
    
    # Process code files
    suggestions = {}
    for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith((".py", ".js", ".java")):  # Add more extensions as needed
                file_path = os.path.join(root, file)
                comments = suggest_comments_for_file(file_path)
                if comments:
                    suggestions[file] = comments

    # Clean up cloned repo
    shutil.rmtree(repo_path)

    # Format and return suggestions
    if suggestions:
        result = "\n\n".join([f"File: {file}\n{comments}" for file, comments in suggestions.items()])
        return result
    else:
        return "No comments or docstrings were suggested for the files in the repository."

In [30]:
# Gradio UI
def main(repo_url):
    return process_repo(repo_url)

In [31]:
ui = gr.Interface(
    fn=main,
    inputs=gr.Textbox(label="GitHub Repository Link", placeholder="Enter public GitHub repo link here..."),
    outputs=gr.Textbox(label="Suggested Comments/Docstrings"),
    title="GitHub Repo Code Commenting Tool",
    description="Enter a public GitHub repository link. The tool will analyze the code and suggest comments/docstrings for the files in the repository."
)

In [32]:
# Launch the Gradio app
if __name__ == "__main__":
    ui.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [33]:
ui.close()

Closing server running on port: 7861
